# ETL - Fase 4: Consolidamos los df para alimentar las consultas
En esta seccion, consolidamos lo df generados en las fases anteriores de ETL. De igual manera, creamos dos subconjuntos de datos para alimentar los principales dos tipos de consulta: las funciones generales y el sistema de recomendacion (modelo ML).

In [1]:
import os
import pandas as pd

In [2]:
data_movies_subset = pd.read_csv(
    os.path.join("2_pipeline","data_movies_subset_limpia.csv"),index_col=0).convert_dtypes() # output paso 01
data_movies_normalizada = pd.read_csv(
    os.path.join("2_pipeline","data_movies_normalizada.csv"),index_col=0).convert_dtypes() # output paso 02
data_credits_normalizada = pd.read_csv(
    os.path.join("2_pipeline","credits_normalizada.csv"),index_col=0).convert_dtypes() # output paso 03

In [3]:
data_movies_subset.info(verbose=True)
data_movies_subset.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45346 non-null  Int64  
 1   pelicula_id        45346 non-null  Int64  
 2   original_language  45335 non-null  string 
 3   overview           44405 non-null  string 
 4   popularity         45346 non-null  Float64
 5   release_date       45346 non-null  string 
 6   revenue            45346 non-null  Int64  
 7   runtime            45100 non-null  Int64  
 8   title              45346 non-null  string 
 9   vote_average       45346 non-null  Float64
 10  vote_count         45346 non-null  Int64  
 11  release_year       45346 non-null  Int64  
 12  return             45346 non-null  Float64
dtypes: Float64(3), Int64(6), string(4)
memory usage: 5.2 MB


budget                 0
pelicula_id            0
original_language     11
overview             941
popularity             0
release_date           0
revenue                0
runtime              246
title                  0
vote_average           0
vote_count             0
release_year           0
return                 0
dtype: int64

In [4]:
data_movies_normalizada.info(verbose=True)
data_movies_normalizada.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 55897 entries, 0 to 55896
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   pelicula_id               55897 non-null  Int64 
 1   franquicia_id             55897 non-null  Int64 
 2   franquicia                55897 non-null  string
 3   productora                53879 non-null  string
 4   productora_id             53879 non-null  Int64 
 5   genres_id                 55754 non-null  Int64 
 6   genres                    55754 non-null  string
 7   pais_isocode              55285 non-null  string
 8   pais_name                 55285 non-null  string
 9   spoken_languages_isocode  55728 non-null  string
 10  spoken_languages_name     55355 non-null  string
dtypes: Int64(4), string(7)
memory usage: 5.3 MB


pelicula_id                    0
franquicia_id                  0
franquicia                     0
productora                  2018
productora_id               2018
genres_id                    143
genres                       143
pais_isocode                 612
pais_name                    612
spoken_languages_isocode     169
spoken_languages_name        542
dtype: int64

In [5]:
data_credits_normalizada.info(verbose=True)
data_credits_normalizada.isna().sum()
data_credits_normalizada.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 106173 entries, 0 to 106172
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   pelicula_id           106173 non-null  Int64 
 1   prtgnst_name          106173 non-null  string
 2   prtgnst_gender_strng  106173 non-null  string
 3   prtgnst_nivel         106173 non-null  Int64 
 4   director              106138 non-null  string
 5   executive_producer    38468 non-null   string
dtypes: Int64(2), string(4)
memory usage: 5.9 MB


,pelicula_id,prtgnst_name,prtgnst_gender_strng,prtgnst_nivel,director,executive_producer
0,862,Tom Hanks,hombre,1,John Lasseter,Ed Catmull
1,862,Tom Hanks,hombre,1,John Lasseter,Steve Jobs
2,862,Tim Allen,hombre,2,John Lasseter,Ed Catmull
3,862,Tim Allen,hombre,2,John Lasseter,Steve Jobs
4,8844,Robin Williams,hombre,1,Joe Johnston,Larry J. Franco
5,8844,Robin Williams,hombre,1,Joe Johnston,Ted Field
6,8844,Robin Williams,hombre,1,Joe Johnston,Robert W. Cort
7,8844,Jonathan Hyde,hombre,2,Joe Johnston,Larry J. Franco
8,8844,Jonathan Hyde,hombre,2,Joe Johnston,Ted Field
9,8844,Jonathan Hyde,hombre,2,Joe Johnston,Robert W. Cort


## 4.1 Analisis de disponibilidad de datos


En esta seccion examinamos la disponibilidad de datos para nuestro universo de casos: peliculas. Conocer la disponibilidad de datos es indispensable por dos razones:

1. Determina la naturaleza de los joins: Un paso previo a la union de dfs es determinar cual de los dfs contiene la mayor cantidad de informacion disponible para nuestro universo de casos: peliculas. Esto determina la naturaleza del join; es decir, si hacemos un inner o un left join. En caso que un df contenga informacion sobre un mayor numero de casos que el otro, hacemos un left join con el fin garantizar el mayor numero de registros posible.

2. Afecta el desempeño de las consultas: diferencias en las disponibilidad de informacion puede afectar el desempeno de unas consultas con respecto a otras; es decir, mientras que algunas consultas pueden gozar de una gran disponibilidad de informacion, otras pueden verse seriamente afectadas por informacion limitada. Conocer esta ventajas y limitaciones nos permite anticipar futuros problemas en el desempeno de las consultas.

### 4.1.1 Descripcion de disponibilidad de informacion

In [6]:
# evaluamos cual df contiene informacion para el mayor numero de casos.
movies_subset_peliculas=data_movies_subset['pelicula_id'].nunique()
movies_normalizada_peliculas=data_movies_normalizada['pelicula_id'].nunique()
credits_normalizada_peliculas=data_credits_normalizada['pelicula_id'].nunique()
print("Universo de pelicula unicas, segun df de referencia:",movies_subset_peliculas)
print("Universo de pelicula unicas, segun movies_normalizada:",movies_normalizada_peliculas)
print("Universo de pelicula unicas, segun credits_normalizada:",credits_normalizada_peliculas)
print("Porcentaje de valores nulos en columnas de movies_normalizada del total de registros (df referencia):",
    round((1-(movies_normalizada_peliculas/movies_subset_peliculas))*100,2),"%")
print("Porcentaje de valores nulos en columnas de credits_normalizada del total de registros (df referencia):",
    round((1-(credits_normalizada_peliculas/movies_subset_peliculas))*100,2),"%")

Universo de pelicula unicas, segun df de referencia: 45346
Universo de pelicula unicas, segun movies_normalizada: 4487
Universo de pelicula unicas, segun credits_normalizada: 42317
Porcentaje de valores nulos en columnas de movies_normalizada del total de registros (df referencia): 90.1 %
Porcentaje de valores nulos en columnas de credits_normalizada del total de registros (df referencia): 6.68 %


Con base en lo anterior, la base de datos final despues del merge tendria la siguientes caracteristicas:
Todos las peliculas tendria valores nulos en alguna columna, puesto que el df de referencia contiene informacion sobre un mayor numero de peliculas en comparacion a los otros dos dfs. Despues del merge, las columnas con mayor numero de valores nulos serian las columnas que corresponden al df `data_movies_normalizada`; es este caso, el dataset final tendria valores nulos para un 90% de las peliculas.

### 4.1.2 Implicaciones de disponibilidad de datos para las consultas

Las funciones que mas se afectarian por esta baja disponibilidad de datos seria las siguientes consultas:

- CUALES?????

Con base en lo anterior, desarrollamos un estrategia de imputacion de valores nulos.

Lo anterior debe orientar nuestro diseno del modelo de ML. Un modelo que haga uso de las columnas que corresponden a data_movies_normalizada (i.e., franquicia, productora, el genero de la pelicula (genres), e idiomas hablados (spoken_languages_name)), estaria desaprovechando la disponbilidad de datos, puesto que solo tendria informacion para el ~10% de universo de casos en nuestra base de dato final. Por tal motivo, lo mejor seria hacer uso la informacion contenida en data_movies_subset y en data_credits_normalizada al diseñar el modelo de ML.

### 4.1.3. Imputacion de valores nulos

In [7]:
# exportamos el df final
data_mvp_final_normalizada = pd.merge(
    pd.merge(data_movies_subset, # tomamos como principal referencia el df con el mayor numero de registros
    data_movies_normalizada,
    how='left',
    on=['pelicula_id']),
    data_credits_normalizada, # tomamos como ultima referencia el df con el menor numero de registros
    how='left',
    on=['pelicula_id'])
#data_mvp_final_normalizada.info()
data_mvp_final_normalizada.isna().sum()

budget                           0
pelicula_id                      0
original_language               16
overview                      1756
popularity                       0
release_date                     0
revenue                          0
runtime                        368
title                            0
vote_average                     0
vote_count                       0
release_year                     0
return                           0
franquicia_id                96004
franquicia                   96004
productora                  104843
productora_id               104843
genres_id                    96277
genres                       96277
pais_isocode                 97396
pais_name                    97396
spoken_languages_isocode     96427
spoken_languages_name        98079
prtgnst_name                  3378
prtgnst_gender_strng          3378
prtgnst_nivel                 3378
director                      3443
executive_producer          142500
dtype: int64

In [8]:
# Evaluamos la cantidad de peliculas sin idioma
print("Numero de peliculas sin idioma:",data_mvp_final_normalizada.loc[
    data_mvp_final_normalizada['original_language'].isna()==True]['pelicula_id'].nunique())

# guardamos la posicion de las filas para comparar el resultado de la imputacion
language_null_index=data_mvp_final_normalizada.loc[
    data_mvp_final_normalizada['original_language'].isna()==True].index

# visualizamos
data_mvp_final_normalizada.loc[
    data_mvp_final_normalizada['original_language'].isna()==True].head(3)


Numero de peliculas sin idioma: 11


,budget,pelicula_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,...,genres,pais_isocode,pais_name,spoken_languages_isocode,spoken_languages_name,prtgnst_name,prtgnst_gender_strng,prtgnst_nivel,director,executive_producer
194537,0,283101,<NA>,Documentary about the production of The Third ...,0.017007,2004-10-11,0,95,Shadowing the Third Man,0.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,John Hurt,hombre,2,Frederick Baker,<NA>
212270,0,103902,<NA>,An Outback farmer takes in an Afghani woman wh...,0.359818,2007-08-04,0,94,Unfinished Sky,6.4,...,<NA>,<NA>,<NA>,<NA>,<NA>,William McInnes,hombre,1,Peter Duncan,<NA>
212271,0,103902,<NA>,An Outback farmer takes in an Afghani woman wh...,0.359818,2007-08-04,0,94,Unfinished Sky,6.4,...,<NA>,<NA>,<NA>,<NA>,<NA>,Monic Hendrickx,mujer,2,Peter Duncan,<NA>


In [9]:
# Imputamos en idioma con base en el titulo de la pelicula
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
print("Ejemplo de identificacion de idioma para 'Lettre d'une inconnue':",detect("Lettre d'une inconnue"))

language_fill = data_mvp_final_normalizada.loc[
    data_mvp_final_normalizada['original_language'].isna()==True, 'title'].apply(detect) # aplicamos la funcion de identificacion de idiomas a los casos nulos
data_mvp_final_normalizada.loc[
    data_mvp_final_normalizada['original_language'].isnull() & (data_mvp_final_normalizada['original_language'].isna()==True), # condiciones
    'original_language'] = language_fill # imputamos los valores al base de datos

# visualizamos los cambios realizados
data_mvp_final_normalizada[['pelicula_id','title','original_language']].loc[language_null_index].drop_duplicates()

Ejemplo de identificacion de idioma para 'Lettre d'une inconnue': fr


,pelicula_id,title,original_language
194537,283101,Shadowing the Third Man,en
212270,103902,Unfinished Sky,en
219025,359195,13 Fighting Men,en
269410,147050,Lambchops,ca
284685,257095,Prince Bayaya,tr
296700,332742,Song of Lahore,en
299040,144410,Annabelle Serpentine Dance,fr
306165,380438,Lettre d'une inconnue,fr
307123,381096,Yarn,tr
307598,381525,WiNWiN,sw


In [10]:
# imputamos valores nulos en runtime usando la mediana, puesto que es mas resistente a la influencia de outliers
imp_median_runtime=data_mvp_final_normalizada[['runtime']].drop_duplicates().dropna().median()
data_mvp_final_normalizada[['runtime']] = data_mvp_final_normalizada[['runtime']].fillna(imp_median_runtime)

In [11]:
# asumimos que los valores nulos en franquicia corresponden a peliculas que no pertnenecen a ninguna franquicia.
# Con base en lo anterior, imputamos el valor 'Sin franquicia' para los casos nulos
data_mvp_final_normalizada['franquicia'].fillna('Sin franquicia', inplace = True)

columns_desconocido=['productora','genres','pais_name','spoken_languages_name','director']
data_mvp_final_normalizada.loc[:,columns_desconocido]=data_mvp_final_normalizada.loc[:,columns_desconocido].fillna('desconocido')

In [12]:
# Nuestros resultados despues de la primera fase de imputacion
#data_mvp_final_normalizada.info()
data_mvp_final_normalizada.isna().sum()

budget                           0
pelicula_id                      0
original_language                0
overview                      1756
popularity                       0
release_date                     0
revenue                          0
runtime                          0
title                            0
vote_average                     0
vote_count                       0
release_year                     0
return                           0
franquicia_id                96004
franquicia                       0
productora                       0
productora_id               104843
genres_id                    96277
genres                           0
pais_isocode                 97396
pais_name                        0
spoken_languages_isocode     96427
spoken_languages_name            0
prtgnst_name                  3378
prtgnst_gender_strng          3378
prtgnst_nivel                 3378
director                         0
executive_producer          142500
dtype: int64

## 4.2 Generacion de dfs para alimentar las consultas

Separamos el df final en dos subconjuntos:
1. df para alimentar las funciones
2. df para alimentar el modelo ml y el sistema de recomendacion.

### 4.2.1. Subconjunto de datos para alimentar las funciones

In [13]:
# primer subconjunto: df para alimentar las funciones
columns_funciones=[
    'pelicula_id', # todas
    'title', # varias
    'original_language', # def peliculas_idioma( Idioma: str )
    'runtime', # def peliculas_duracion( Pelicula: str )
    'release_date', # def peliculas_duracion( Pelicula: str )
    'franquicia', # def franquicia( Franquicia: str )
    'budget', # def franquicia( Franquicia: str )
    'revenue', # def franquicia( Franquicia: str )
    'pais_name', # def peliculas_pais( Pais: str )
    'productora', # productoras_exitosas( Productora: str ),
    'director', # get_director( nombre_director )
    'return', # get_director( nombre_director )
    'release_year' # get_director( nombre_director )
    ]

In [14]:
data_mvp_funciones=data_mvp_final_normalizada.loc[:,columns_funciones].drop_duplicates()
print("Universo de peliculas unicas en el df que alimenta las funciones:",
      data_mvp_funciones['pelicula_id'].nunique())

Universo de peliculas unicas en el df que alimenta las funciones: 45346


In [15]:
data_mvp_funciones.to_csv(os.path.join("3_output","data_mvp_final_funciones.csv"))
data_mvp_funciones.info()
data_mvp_funciones.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 59545 entries, 0 to 310039
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        59545 non-null  Int64  
 1   title              59545 non-null  string 
 2   original_language  59545 non-null  string 
 3   runtime            59545 non-null  Int64  
 4   release_date       59545 non-null  string 
 5   franquicia         59545 non-null  string 
 6   budget             59545 non-null  Int64  
 7   revenue            59545 non-null  Int64  
 8   pais_name          59545 non-null  string 
 9   productora         59545 non-null  string 
 10  director           59545 non-null  string 
 11  return             59545 non-null  Float64
 12  release_year       59545 non-null  Int64  
dtypes: Float64(1), Int64(5), string(7)
memory usage: 6.7 MB


pelicula_id          0
title                0
original_language    0
runtime              0
release_date         0
franquicia           0
budget               0
revenue              0
pais_name            0
productora           0
director             0
return               0
release_year         0
dtype: int64

### 4.2.3. Subconjunto de datos para alimentar la consulta de director

In [16]:
# primero, calculamos el retorno por director
data_mvp_funciones_exitodir=data_mvp_funciones[['director','revenue','budget']].loc[
    (data_mvp_funciones['revenue']!=0) & (data_mvp_funciones['budget']!=0) # filtramos unicamente aquellas filas para las que hay informacion
].drop_duplicates()
print("Numero de directores con informacion disponible", data_mvp_funciones_exitodir['director'].nunique())
data_exitodir_revenue=data_mvp_funciones_exitodir[['director','revenue']].groupby(['director']).sum()
data_exitodir_budget=data_mvp_funciones_exitodir[['director','budget']].groupby(['director']).sum()
data_mvp_funciones_exitodir=pd.merge(
    data_exitodir_revenue,
    data_exitodir_budget,
    on=['director'])
data_mvp_funciones_exitodir['return']=data_mvp_funciones_exitodir['revenue']/data_mvp_funciones_exitodir['budget']
data_mvp_funciones_exitodir.rename(
    columns={'return': 'director_return',
             'revenue':'director_revenue',
             'budget':'director_budget'},
    inplace=True)

# exportamos
data_mvp_funciones_exitodir.reset_index(inplace=True)
data_mvp_funciones_exitodir.info()

Numero de directores con informacion disponible 2617
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2617 entries, 0 to 2616
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   director          2617 non-null   string 
 1   director_revenue  2617 non-null   Int64  
 2   director_budget   2617 non-null   Int64  
 3   director_return   2617 non-null   Float64
dtypes: Float64(1), Int64(2), string(1)
memory usage: 89.6 KB


In [17]:
# segundo subconjunto: df para alimentar las funciones
columns_exitodir=[
    'pelicula_id', # todas
    'title', # varias
    'release_date', # def peliculas_duracion( Pelicula: str )
    'release_year',
    'budget', # def franquicia( Franquicia: str )
    'revenue', # def franquicia( Franquicia: str )
    'return', # get_director( nombre_director )
    'director', # get_director( nombre_director )
    ]

In [18]:
data_mvp_funciones_exitodir_2join=data_mvp_final_normalizada.loc[:,columns_exitodir].drop_duplicates()
data_mvp_exitodir=data_mvp_funciones_exitodir.merge(
    data_mvp_funciones_exitodir_2join,
    on=['director']
)

data_mvp_exitodir.to_csv(os.path.join("3_output","data_mvp_final_funciones_exitodir.csv"))
data_mvp_exitodir.info()
data_mvp_exitodir.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18614 entries, 0 to 18613
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   director          18614 non-null  string 
 1   director_revenue  18614 non-null  Int64  
 2   director_budget   18614 non-null  Int64  
 3   director_return   18614 non-null  Float64
 4   pelicula_id       18614 non-null  Int64  
 5   title             18614 non-null  string 
 6   release_date      18614 non-null  string 
 7   release_year      18614 non-null  Int64  
 8   budget            18614 non-null  Int64  
 9   revenue           18614 non-null  Int64  
 10  return            18614 non-null  Float64
dtypes: Float64(2), Int64(6), string(3)
memory usage: 1.7 MB


director            0
director_revenue    0
director_budget     0
director_return     0
pelicula_id         0
title               0
release_date        0
release_year        0
budget              0
revenue             0
return              0
dtype: int64

### 4.2.2. Subconjunto de datos para alimentar el modelo ML

In [19]:
# segundo subconjunto: df para alimentar el modelo de ML
columns_ml=[
    'pelicula_id',
    'title',
    'popularity',
    'vote_average',
    'release_year',
    'genres',
    'pais_name',
    'franquicia',
    'original_language',
    'prtgnst_gender_strng',
    'prtgnst_nivel',
    'director',
    'executive_producer',
    'overview'
    ]

# aprovechamos las columnas ya limpias en data_mvp_funciones que tambien estan en columns_ml:
columns_funciones_2join=list(set(columns_ml) & set(columns_funciones))
print(columns_funciones_2join)
# tambien incluimos las columnas en columns_ml que no han sido limpiadas:
columns_ml_2join=list(['pelicula_id']+list(set(columns_ml) - set(columns_funciones)))
print(columns_ml_2join)

['pelicula_id', 'franquicia', 'title', 'pais_name', 'release_year', 'director', 'original_language']
['pelicula_id', 'prtgnst_nivel', 'vote_average', 'popularity', 'prtgnst_gender_strng', 'overview', 'genres', 'executive_producer']


In [20]:
# subconjunto de datos para modelo ml
data_mvp_ml=pd.merge(
      data_mvp_funciones.loc[:,columns_funciones_2join].drop_duplicates(),
      data_mvp_final_normalizada.loc[:,columns_ml_2join].drop_duplicates(),
      on='pelicula_id',
      how='left')
print("Universo de peliculas unicas en el df que alimenta el modelo ml:",
      data_mvp_funciones['pelicula_id'].nunique())

Universo de peliculas unicas en el df que alimenta el modelo ml: 45346


In [21]:
# identificamos las columnas con nulos
data_mvp_ml.info()
data_mvp_ml.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140336 entries, 0 to 140335
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   pelicula_id           140336 non-null  Int64  
 1   franquicia            140336 non-null  string 
 2   title                 140336 non-null  string 
 3   pais_name             140336 non-null  string 
 4   release_year          140336 non-null  Int64  
 5   director              140336 non-null  string 
 6   original_language     140336 non-null  string 
 7   prtgnst_nivel         137208 non-null  Int64  
 8   vote_average          140336 non-null  Float64
 9   popularity            140336 non-null  Float64
 10  prtgnst_gender_strng  137208 non-null  string 
 11  overview              138654 non-null  string 
 12  genres                140336 non-null  string 
 13  executive_producer    55985 non-null   string 
dtypes: Float64(2), Int64(3), string(9)
memory usage: 15.

pelicula_id                 0
franquicia                  0
title                       0
pais_name                   0
release_year                0
director                    0
original_language           0
prtgnst_nivel            3128
vote_average                0
popularity                  0
prtgnst_gender_strng     3128
overview                 1682
genres                      0
executive_producer      84351
dtype: int64

#### 4.2.2.2. Imputacion de valores y creacion del grupo 'peliculas de baja circulacion'

Nos concentramos en las columnas restantes que contienen nulos.

In [22]:
columns_fillna_num=[
    'prtgnst_nivel']
columns_fillna_cat=[
    'prtgnst_gender_strng',
    'executive_producer']
data_mvp_ml[columns_fillna_num+columns_fillna_cat].isna().sum()

prtgnst_nivel            3128
prtgnst_gender_strng     3128
executive_producer      84351
dtype: int64

Consideramos las peliculas que contienen valores nulos en estas columnas pertenecen a un grupo: peliculas de baja circulacion. El website the IMDB señala que su fuente de datos no es unicamente estudios de produccion y realizadores de cine, sino que "[...] la mayor parte de nuestra información procede de profesionales y visitantes" ([IMDB](https://help.imdb.com/article/imdb/general-information/where-does-the-information-on-imdb-come-from/GGD7NGF5X3ECFKNN?ref_=helpart_nav_17#)). Por tanto, consideramos que si una pelicula no contiene informacion en la base de datos IMDB es por que pertenece a un grupo que denominamos "peliculas de baja circulacion". Por tal motivo, imputamos el valor `baja_circulacion` en los casos `NA` para asociar la membresia de una pelicula con dicho grupo.

A su vez, la membresia al grupo "peliculas de baja circulacion" ayuda a capturar diferentes intensidades de membresia. Por ejemplo, una pelicula podria tener valores nulos en solo un par de columnas, mientras que otra pelicula podria tener valores nulos en varias columnas; la interseccion de `baja_circulacion` en dos columnas señalaria un caso de bajo grado de "poca circulacion", mientras que la interseccion de varias columnas con el valor `baja_circulacion` señalaria un alto grado de membresia al grupo "peliculas de baja circulacion". De esta manera, incorporamos a nuestro modelo ML no solo la membresia al grupo "peliculas de baja circulacion" sino tambien diferentes intensidades de membresia, permitiendo la identificacion de clusters que de otra manera (p. ej., eliminando los valores nulos) no seria posible.

In [23]:
# algunos casos de peliculas que pertenecen al grupo "peliculas con baja circulacion"
index_fillna_cat=data_mvp_ml.loc[data_mvp_ml['executive_producer'].isna()==True].index # guardamos para comparar
data_mvp_ml[['pelicula_id','title']+columns_fillna_cat].loc[data_mvp_ml['executive_producer'].isna()==True].head(3)


,pelicula_id,title,prtgnst_gender_strng,executive_producer
18,15602,Grumpier Old Men,hombre,<NA>
19,15602,Grumpier Old Men,hombre,<NA>
20,15602,Grumpier Old Men,hombre,<NA>


In [24]:
# inputamos valores nulos con las columnas seleccionadas con el valor 'baja_circulacion';
# cambiamos prtgnst_nivel a string para facilitar a imputacion de valores
#data_mvp_ml['prtgnst_nivel']=data_mvp_ml['prtgnst_nivel'].astype('string')
data_mvp_ml[columns_fillna_cat]=data_mvp_ml[columns_fillna_cat].fillna('desconocido')
data_mvp_ml[['pelicula_id','title']+columns_fillna_cat].loc[index_fillna_cat].head(3)

,pelicula_id,title,prtgnst_gender_strng,executive_producer
18,15602,Grumpier Old Men,hombre,desconocido
19,15602,Grumpier Old Men,hombre,desconocido
20,15602,Grumpier Old Men,hombre,desconocido


In [25]:
data_mvp_ml[columns_fillna_num].drop_duplicates()

,prtgnst_nivel
0,1
2,2
494,<NA>


In [26]:
# inputamos valores nulos con las columnas seleccionadas con el valor 'baja_circulacion';
# cambiamos prtgnst_nivel a string para facilitar a imputacion de valores
#data_mvp_ml['prtgnst_nivel']=data_mvp_ml['prtgnst_nivel'].astype('string')
data_mvp_ml[columns_fillna_num]=data_mvp_ml[columns_fillna_num].fillna(3)
data_mvp_ml[columns_fillna_num].drop_duplicates()

,prtgnst_nivel
0,1
2,2
494,3


In [27]:
# El resultado despues de completar todas las fases de imputacion: exportamos el df para alimentar el modelo ML
data_mvp_ml.to_csv(os.path.join("3_output","data_mvp_final_ml.csv"))
data_mvp_ml.info()
data_mvp_ml.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140336 entries, 0 to 140335
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   pelicula_id           140336 non-null  Int64  
 1   franquicia            140336 non-null  string 
 2   title                 140336 non-null  string 
 3   pais_name             140336 non-null  string 
 4   release_year          140336 non-null  Int64  
 5   director              140336 non-null  string 
 6   original_language     140336 non-null  string 
 7   prtgnst_nivel         140336 non-null  Int64  
 8   vote_average          140336 non-null  Float64
 9   popularity            140336 non-null  Float64
 10  prtgnst_gender_strng  140336 non-null  string 
 11  overview              138654 non-null  string 
 12  genres                140336 non-null  string 
 13  executive_producer    140336 non-null  string 
dtypes: Float64(2), Int64(3), string(9)
memory usage: 15.

pelicula_id                0
franquicia                 0
title                      0
pais_name                  0
release_year               0
director                   0
original_language          0
prtgnst_nivel              0
vote_average               0
popularity                 0
prtgnst_gender_strng       0
overview                1682
genres                     0
executive_producer         0
dtype: int64